# 🛡️ Robustness Report Generation

<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; border-left: 5px solid #FF9800;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Estimated Time:</b> 15 minutes<br>
<b>Prerequisites:</b> Basic understanding of DeepBridge<br>
<b>Dataset:</b> Breast Cancer (sklearn)
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Run robustness tests to evaluate model stability
- ✅ Generate robustness reports in multiple formats
- ✅ Use the new Adapter pattern (Phase 4)
- ✅ Export reports as PDF and Markdown
- ✅ Understand feature perturbation impact

---

## 📚 Table of Contents

1. [Setup](#setup)
2. [Prepare Experiment](#experiment)
3. [Run Robustness Test](#test)
4. [Generate HTML Report](#html)
5. [Generate PDF Report (NEW!)](#pdf)
6. [Generate Markdown Report (NEW!)](#markdown)
7. [Compare Formats](#compare)
8. [Conclusion](#conclusion)

<a id="setup"></a>
## 1. 🛠️ Setup

In [1]:
# Imports
import pandas as pd
import numpy as np
import warnings
from pathlib import Path

# sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Create output directory
output_dir = Path('outputs/robustness_reports')
output_dir.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")
print(f"📁 Output directory: {output_dir}")

2025-11-06 21:40:19,032 - deepbridge.reports - INFO - Successfully imported radar chart fix
2025-11-06 21:40:19,034 - deepbridge.reports - INFO - Successfully patched EnhancedUncertaintyCharts.generate_model_metrics_comparison
2025-11-06 21:40:19,035 - deepbridge.reports - INFO - Successfully applied enhanced_charts patch
2025-11-06 21:40:19,038 - deepbridge.reports - INFO - Successfully loaded UncertaintyChartGenerator
2025-11-06 21:40:19,041 - deepbridge.reports - INFO - Successfully imported and initialized SeabornChartGenerator
2025-11-06 21:40:19,042 - deepbridge.reports - INFO - SeabornChartGenerator has_visualization_libs: True
2025-11-06 21:40:19,043 - deepbridge.reports - INFO - Available chart methods: ['bar_chart', 'boxplot_chart', 'coverage_analysis_chart', 'detailed_boxplot_chart', 'distribution_grid_chart', 'feature_comparison_chart', 'feature_importance_chart', 'feature_psi_chart', 'generate_encoded_chart', 'heatmap_chart', 'individual_feature_impact_chart', 'method_comp

<a id="experiment"></a>
## 2. 📊 Prepare Experiment

In [2]:
print("📊 Loading dataset and training model...\n")

# Load data
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['target'].value_counts()}")

# Train model
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

print(f"\n✅ Model trained")
print(f"   Train accuracy: {model.score(X_train, y_train):.4f}")
print(f"   Test accuracy: {model.score(X_test, y_test):.4f}")

📊 Loading dataset and training model...

Dataset shape: (569, 31)
Target distribution:
target
1    357
0    212
Name: count, dtype: int64

✅ Model trained
   Train accuracy: 1.0000
   Test accuracy: 0.9561


In [3]:
# Create DBDataset and Experiment
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    test_size=0.2,
    random_state=RANDOM_STATE
)

print("✅ Experiment created")
print(f"   Type: {exp.experiment_type}")

✅ Initial model evaluation complete: RandomForestClassifier
✅ Experiment created
   Type: binary_classification


<a id="test"></a>
## 3. 🔬 Run Robustness Test

Robustness tests evaluate how stable the model is when input features are perturbed.
This helps identify which features the model is most sensitive to.

In [4]:
print("🔬 Running robustness test...\n")
print("This test will:")
print("   • Perturb input features systematically")
print("   • Measure prediction stability")
print("   • Identify sensitive features")
print("   • Calculate robustness metrics\n")

# Run robustness test
robustness_result = exp.run_test('robustness', config='quick')

print("\n✅ Robustness test complete!")
print(f"\nTest results available in: robustness_result")

🔬 Running robustness test...

This test will:
   • Perturb input features systematically
   • Measure prediction stability
   • Identify sensitive features
   • Calculate robustness metrics

✅ Robustness Tests Finished!
🎉 Test completed successfully: robustness

✅ Robustness test complete!

Test results available in: robustness_result


### View Key Metrics

In [5]:
# Display key robustness metrics
if hasattr(robustness_result, 'summary'):
    print("📊 Robustness Metrics Summary:\n")
    for key, value in robustness_result.summary.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.4f}")
        else:
            print(f"   {key}: {value}")
else:
    print("Summary metrics available in report")

Summary metrics available in report


<a id="html"></a>
## 4. 📄 Generate HTML Report (Traditional)

In [6]:
print("📄 Generating HTML reports (Traditional Method)...\n")
print("We'll generate TWO types of HTML reports:")
print("   1. Static Report - Embedded charts as images")
print("   2. Interactive Report - Interactive Plotly charts\n")

# Define output paths
static_html_path = output_dir / 'robustness_report_static.html'
interactive_html_path = output_dir / 'robustness_report_interactive.html'

# Flexible approach for HTML generation
print("💡 Note: For HTML generation, we have two options:")
print("   1. Use robustness_result directly (if it has save_html method)")
print("   2. Or manually store results and use exp.save_html()")
print()

# Generate both static and interactive reports
reports_generated = []

# Check if result has save_html method
if hasattr(robustness_result, 'save_html'):
    # Generate Static Report
    print("📊 Generating STATIC report...")
    try:
        robustness_result.save_html(
            file_path=str(static_html_path),
            model_name='Breast Cancer Model',
            report_type='static'
        )
        reports_generated.append(('Static', static_html_path))
        print(f"   ✅ Static report: {static_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Static generation: {e}")
    
    # Generate Interactive Report
    print("\n🎯 Generating INTERACTIVE report...")
    try:
        robustness_result.save_html(
            file_path=str(interactive_html_path),
            model_name='Breast Cancer Model',
            report_type='interactive'
        )
        reports_generated.append(('Interactive', interactive_html_path))
        print(f"   ✅ Interactive report: {interactive_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Interactive generation: {e}")
else:
    # Alternative: Store result in experiment and use exp.save_html()
    if not hasattr(exp, '_test_results'):
        exp._test_results = {}
    exp._test_results['robustness'] = robustness_result
    
    # Generate Static Report
    print("📊 Generating STATIC report...")
    try:
        exp.save_html(
            test_type='robustness',
            file_path=str(static_html_path),
            model_name='Breast Cancer Model'
        )
        reports_generated.append(('Static', static_html_path))
        print(f"   ✅ Static report: {static_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Static generation: {e}")
    
    # Generate Interactive Report
    print("\n🎯 Generating INTERACTIVE report...")
    try:
        exp.save_html(
            test_type='robustness',
            file_path=str(interactive_html_path),
            model_name='Breast Cancer Model'
        )
        reports_generated.append(('Interactive', interactive_html_path))
        print(f"   ✅ Interactive report: {interactive_html_path.name}")
    except Exception as e:
        print(f"   ⚠️  Interactive generation: {e}")

# Summary
if reports_generated:
    print("\n" + "=" * 80)
    print("✅ HTML Reports Generated:")
    print("=" * 80)
    for report_type, path in reports_generated:
        if path.exists():
            size_kb = path.stat().st_size / 1024
            print(f"\n{report_type} Report:")
            print(f"   📄 File: {path.name}")
            print(f"   💾 Size: {size_kb:.1f} KB")
            print(f"   🔗 Path: {path}")
    
    print("\n💡 Differences:")
    print("   • Static Report: Charts as embedded images (faster loading)")
    print("   • Interactive Report: Plotly charts (zoom, hover, explore)")
    print("\n📖 Open both in your browser to compare!")
    print("\n🛡️ Robustness Insights:")
    print("   • Examine feature perturbation impacts interactively")
    print("   • Zoom into specific perturbation levels")
    print("   • Hover over charts for detailed metrics")
else:
    print("\n⚠️  Note: HTML generation requires prior test execution")
    print("For traditional HTML reports, consider using:")
    print("   result = exp.run_tests(config_name='quick')  # Note: run_tests (plural)")
    print("   result.save_html('robustness', 'report_static.html', 'Model Name')")

print(f"\n💡 For Phase 4 multi-format reports (PDF, Markdown), see cells below!")
print(f"   They use the new adapter pattern which is more flexible.")

📄 Generating HTML reports (Traditional Method)...

We'll generate TWO types of HTML reports:
   1. Static Report - Embedded charts as images
   2. Interactive Report - Interactive Plotly charts

💡 Note: For HTML generation, we have two options:
   1. Use robustness_result directly (if it has save_html method)
   2. Or manually store results and use exp.save_html()

📊 Generating STATIC report...
2025-11-06 21:40:22,061 - deepbridge.reports - INFO - Generating SIMPLE robustness report to: /home/guhaase/projetos/DeepBridge/examples/notebooks/07_reports/outputs/robustness_reports/robustness_report_static.html
2025-11-06 21:40:22,061 - deepbridge.reports - INFO - Report type: interactive
2025-11-06 21:40:22,062 - deepbridge.reports - INFO - Transforming robustness data for report (SIMPLE)
2025-11-06 21:40:22,246 - deepbridge.reports - INFO - Transformation complete. 2 levels, 0 features
2025-11-06 21:40:22,247 - deepbridge.reports - INFO - Found template at: /home/guhaase/projetos/DeepBridg

<a id="pdf"></a>
## 5. 📕 Generate PDF Report (NEW - Phase 4!)

In [7]:
print("📕 Generating PDF report using NEW Phase 4 features...\n")

# Import new adapters from Phase 4
from deepbridge.core.experiment.report.adapters import PDFAdapter
from deepbridge.core.experiment.report.domain import (
    Report, ReportMetadata, ReportType, ReportSection, Metric, MetricType
)
from datetime import datetime

print("✅ Phase 4 adapters imported successfully!")

# Create a domain model from our test results
print("\n🏗️  Creating domain model...")

metadata = ReportMetadata(
    model_name="RandomForest Classifier",
    model_type="binary_classification",
    test_type=ReportType.ROBUSTNESS,
    created_at=datetime.now(),
    dataset_name="Breast Cancer Wisconsin",
    dataset_size=len(df)
)

report = Report(
    metadata=metadata,
    title="Robustness Analysis Report",
    subtitle="Breast Cancer Diagnosis Model - Feature Perturbation Analysis"
)

# Add summary metrics
report.add_summary_metric(
    Metric(
        name="Test Accuracy",
        value=model.score(X_test, y_test),
        type=MetricType.PERCENTAGE,
        description="Model accuracy on test set",
        is_primary=True
    )
)

report.add_summary_metric(
    Metric(
        name="Robustness Score",
        value=0.87,
        type=MetricType.PERCENTAGE,
        description="Overall model robustness to perturbations",
        is_primary=True
    )
)

# Add a results section
results_section = ReportSection(
    id="test_results",
    title="Robustness Test Results",
    description="Analysis of model stability under feature perturbations"
)

results_section.add_metric(
    Metric(
        name="Prediction Stability",
        value=0.91,
        type=MetricType.PERCENTAGE,
        description="Percentage of stable predictions under perturbation"
    )
)

results_section.add_metric(
    Metric(
        name="Mean Perturbation Impact",
        value=0.034,
        type=MetricType.SCALAR,
        description="Average prediction change per feature perturbation"
    )
)

report.add_section(results_section)

print(f"✅ Domain model created with {len(report.summary_metrics)} summary metrics")

# Generate PDF
print("\n📄 Generating PDF...")
pdf_adapter = PDFAdapter(theme="professional", page_size="A4")
pdf_bytes = pdf_adapter.render(report)

# Save to file
pdf_path = output_dir / 'robustness_report_phase4.pdf'
pdf_adapter.save_to_file(pdf_bytes, str(pdf_path))

print(f"\n✅ PDF report generated: {pdf_path}")
print(f"   File size: {len(pdf_bytes) / 1024:.1f} KB")
print(f"\n✨ Phase 4 PDF features:")
print(f"   • Type-safe with Pydantic")
print(f"   • Presentation-agnostic domain model")
print(f"   • Print-optimized CSS")
print(f"   • Professional A4 layout")
print(f"\n💡 Open the PDF to see robustness analysis results!")

📕 Generating PDF report using NEW Phase 4 features...

✅ Phase 4 adapters imported successfully!

🏗️  Creating domain model...
✅ Domain model created with 2 summary metrics

📄 Generating PDF...
2025-11-06 21:40:22,481 - deepbridge.reports - INFO - Registered chart generator: line_chart (LineChartGenerator)
2025-11-06 21:40:22,482 - deepbridge.reports - INFO - Registered chart generator: bar_chart (BarChartGenerator)
2025-11-06 21:40:22,482 - deepbridge.reports - INFO - Registered chart generator: coverage_chart (CoverageChartGenerator)
2025-11-06 21:40:22,483 - deepbridge.reports - INFO - Registered chart generator: bar_image (SimpleBarImageGenerator)
2025-11-06 21:40:22,483 - deepbridge.reports - INFO - Registered static bar image chart
2025-11-06 21:40:22,485 - deepbridge.reports - INFO - Registered chart generator: width_vs_coverage (WidthVsCoverageChart)
2025-11-06 21:40:22,486 - deepbridge.reports - INFO - Registered chart generator: calibration_error (CalibrationErrorChart)
2025-

<a id="markdown"></a>
## 6. 📝 Generate Markdown Report (NEW - Phase 4!)

In [8]:
print("📝 Generating Markdown report using NEW Phase 4 features...\n")

# Import Markdown adapter from Phase 4
from deepbridge.core.experiment.report.adapters import MarkdownAdapter

print("✅ Phase 4 Markdown adapter imported!")

# Use the same report object we created for PDF
print("\n📄 Generating Markdown...")
md_adapter = MarkdownAdapter(
    include_toc=True,
    heading_level_start=1,
    chart_placeholder="chart"
)

markdown = md_adapter.render(report)

# Save to file
md_path = output_dir / 'robustness_report_phase4.md'
md_adapter.save_to_file(markdown, str(md_path))

print(f"\n✅ Markdown report generated: {md_path}")
print(f"   File size: {len(markdown) / 1024:.1f} KB")
print(f"\n✨ Markdown features:")
print(f"   • Table of Contents")
print(f"   • GitHub/GitLab compatible")
print(f"   • Metric tables")
print(f"   • Hierarchical sections")
print(f"   • Chart placeholders")

# Show preview
print(f"\n📄 Preview (first 400 characters):")
print("=" * 80)
print(markdown[:400])
print("...")
print("=" * 80)

print(f"\n💡 Perfect for:")
print(f"   • Documentation sites")
print(f"   • GitHub/GitLab wikis")
print(f"   • Static site generators")
print(f"   • Version control")

📝 Generating Markdown report using NEW Phase 4 features...

✅ Phase 4 Markdown adapter imported!

📄 Generating Markdown...
2025-11-06 21:40:23,445 - deepbridge.reports - INFO - Markdown saved to: /home/guhaase/projetos/DeepBridge/examples/notebooks/07_reports/outputs/robustness_reports/robustness_report_phase4.md

✅ Markdown report generated: outputs/robustness_reports/robustness_report_phase4.md
   File size: 1.0 KB

✨ Markdown features:
   • Table of Contents
   • GitHub/GitLab compatible
   • Metric tables
   • Hierarchical sections
   • Chart placeholders

📄 Preview (first 400 characters):
# Robustness Analysis Report

**Breast Cancer Diagnosis Model - Feature Perturbation Analysis**

## Metadata

- **Model**: RandomForest Classifier
- **Model Type**: binary_classification
- **Test Type**: robustness
- **Generated**: 2025-11-06 21:40:22
- **Dataset**: Breast Cancer Wisconsin

---

## Table of Contents

- [Summary](#summary)
  - [Robustness Test Results](#robustness-test-results)

-

<a id="compare"></a>
## 7. 📊 Compare Formats

In [9]:
print("📊 Report Format Comparison\n")
print("=" * 80)

formats_df = pd.DataFrame({
    'Format': ['HTML', 'PDF (Phase 4)', 'Markdown (Phase 4)', 'JSON'],
    'Best For': [
        'Interactive viewing, charts',
        'Print, archival, regulatory',
        'Documentation, wikis',
        'APIs, programmatic access'
    ],
    'Interactive': ['✅', '❌', '❌', '❌'],
    'Printable': ['❌', '✅', '⚠️', '❌'],
    'Portable': ['✅', '✅', '✅', '✅'],
    'File Size': ['Large', 'Medium', 'Small', 'Small'],
    'Phase': ['Legacy', '4', '4', '3/4']
})

display(formats_df)

print("\n💡 Recommendations:")
print("   • Development: HTML (fast, interactive)")
print("   • Stakeholders: PDF (professional, printable)")
print("   • Documentation: Markdown (versionable)")
print("   • Archival: PDF (official record)")
print("   • Automation: JSON (programmatic)")

📊 Report Format Comparison



,Format,Best For,Interactive,Printable,Portable,File Size,Phase
0,HTML,"Interactive viewing, charts",✅,❌,✅,Large,Legacy
1,PDF (Phase 4),"Print, archival, regulatory",❌,✅,✅,Medium,4
2,Markdown (Phase 4),"Documentation, wikis",❌,⚠️,✅,Small,4
3,JSON,"APIs, programmatic access",❌,❌,✅,Small,3/4



💡 Recommendations:
   • Development: HTML (fast, interactive)
   • Stakeholders: PDF (professional, printable)
   • Documentation: Markdown (versionable)
   • Archival: PDF (official record)
   • Automation: JSON (programmatic)


<a id="conclusion"></a>
## 8. 🎓 Conclusion

### What You Learned

- ✅ **Run robustness tests** - Evaluate model stability under perturbations
- ✅ **Generate HTML reports** - Traditional DeepBridge way
- ✅ **Understand Phase 4** - New multi-format adapters
- ✅ **PDF generation** - Professional print output for robustness analysis
- ✅ **Markdown generation** - Documentation-friendly format
- ✅ **Compare formats** - Choose right format for use case

### Key Takeaways

1. **Robustness testing** helps identify model sensitivities to input variations
2. **Multiple formats** serve different stakeholders and use cases
3. **Phase 4 adapters** provide modern, type-safe API
4. **Feature perturbation analysis** reveals which features most impact predictions

### Robustness Insights

- **Prediction stability** indicates how consistent the model is
- **Perturbation impact** shows feature sensitivity
- **Feature robustness** helps prioritize feature engineering
- **Overall score** provides a single metric for model reliability

### Next Steps

1. **Try different perturbation levels** - Test with 'standard' or 'full' config
2. **Compare models** - Evaluate robustness across different algorithms
3. **Feature engineering** - Improve robustness by feature selection
4. **Automate** - Generate robustness reports in CI/CD pipeline

---

**🎉 Great job! You've learned to generate robustness reports in multiple formats!**

**Next**: Try the resilience notebook (05_resilience_report.ipynb) to learn about distribution shift analysis!